In [1]:
import requests
a = requests.get("https://bullx.io/pump-vision")

# DO EVERYTHING IN HEX STRINGS

In [77]:
def convert_hex(val, base=16):
    
    if type(val) == int:
        return hex(int(val))
    if type(val) == str:
        return hex(int(val, base))



'0x14988ec'

In [2]:
import requests
import json

def fetch_new_crypto_coins():
    # CoinGecko API endpoint for fetching new crypto coins
    url = "https://api.coingecko.com/api/v3/coins/list"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Sort coins by addition date (newest first, if available)
        # Note: The CoinGecko API does not directly provide a creation or addition date,
        # so you might need to filter based on other criteria if required.

        print(f"Fetched {len(data)} cryptocurrencies.")
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

def save_to_file(data, filename="new_crypto_coins.json"):
    with open(filename, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Saved data to {filename}")

d = fetch_new_crypto_coins()


Fetched 16484 cryptocurrencies.


In [ ]:
d[12]['id']

{'id': '0xdao', 'symbol': 'oxd', 'name': '0xDAO'}

In [ ]:
import websocket
import json
import threading

def on_message(ws, message):
    data = json.loads(message)
    symbol = data['s']
    price = data['c']
    print(f"{symbol}: ${price}")

def on_error(ws, error):
    print(f"Error: {error}")

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    # List of cryptocurrency symbols to track (Binance format, e.g., BTCUSDT)
    symbols = ["btcusdt", "ethusdt", "bnbusdt"]  # Add the symbols of the coins you want to track
    for symbol in symbols:
        params = {
            "method": "SUBSCRIBE",
            "params": [f"{symbol}@ticker"],
            "id": 1
        }
        ws.send(json.dumps(params))

def start_websocket():
    websocket_url = "wss://stream.binance.com:9443/ws"
    ws = websocket.WebSocketApp(websocket_url,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()

if __name__ == "__main__":
    print("Starting real-time cryptocurrency price tracking...")
    websocket_thread = threading.Thread(target=start_websocket)
    websocket_thread.start()


- Find new currency to track

- Start tracking 

- Apply (ddy - dy) logic

- Profit?


# Find new currency

In [83]:
import requests

def get_recent_id():
    API_KEY = 'KNQNaxy_OaXlR2VTr6kIrCIjZuyGT0f8'
    CHAIN_ID = '1399811149'
    URL = f'https://eth-mainnet.alchemyapi.io/v2/{API_KEY}'

    # Step 1: Get the latest block number
    payload_block_number = {
        "jsonrpc": "2.0",
        "method": "eth_blockNumber",
        "params": [],
        "id": 1
    }

    response_block_number = requests.post(URL, json=payload_block_number)
    block_number_data = response_block_number.json()
    latest_block_number = int(block_number_data['result'], 16)

    print(f"Latest Block Number: {latest_block_number}")

    # Step 2: Query the latest block for transactions
    payload_block = {
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [hex(latest_block_number), True],  # 'True' to get all transactions in the block
        "id": 1
    }

    response_block = requests.post(URL, json=payload_block)
    block_data = response_block.json()

    # Step 3: Filter for contract creation transactions (looking for 'to' as None)
    token_contracts = []
    for tx in block_data['result']['transactions']:
        if tx['to'] is None:  # Contract creation
            token_contracts.append(tx)

    # Step 4: Print or process the contract creation transactions
    print(f"Found {len(token_contracts)} contract creation transactions.")
    for contract in token_contracts:
        print(f"Contract Address: {contract['hash']} - Created at Block: {contract['blockNumber']}")

def fetch_block(hexID):
    API_KEY = 'KNQNaxy_OaXlR2VTr6kIrCIjZuyGT0f8'
    CHAIN_ID = '1399811149'
    BLOCK_ID = convert_hex(hexID)
    URL = f'https://eth-mainnet.alchemyapi.io/v2/{API_KEY}'

    # Step 1: Get the latest block number
    payload_block_number = {
        "jsonrpc": "2.0",
        "method": "eth_blockNumber",
        "params": [],
        "id": 1
    }

    response_block_number = requests.post(URL, json=payload_block_number)
    block_number_data = response_block_number.json()
    print(f"Most recent {block_number_data}")
    latest_block_number = int(block_number_data['result'], 16)
    print(f"Latest Block Number: {latest_block_number}")

    # # Step 2: Query the latest block for transactions
    payload_block = {
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [BLOCK_ID,True],  # 'True' to get all transactions in the block
        "id": 1
    }

    response_block = requests.post(URL, json=payload_block)
    block_data = response_block.json()
    return block_data
data = fetch_block("0x14988eb")



Most recent {'jsonrpc': '2.0', 'id': 1, 'result': '0x149890e'}
Latest Block Number: 21596430


In [88]:
data

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'number': '0x14988eb',
  'hash': '0xf26f3d8f69ac1ac9e35f954cd442eae6927a77948810b40e231a939c4b6c8917',
  'transactions': [{'blockHash': '0xf26f3d8f69ac1ac9e35f954cd442eae6927a77948810b40e231a939c4b6c8917',
    'blockNumber': '0x14988eb',
    'hash': '0x5a8fb0e0fe183eb3693b39e8c3aa79fcadd11620e4443227e89756ad321c2920',
    'yParity': '0x1',
    'accessList': [],
    'transactionIndex': '0x0',
    'type': '0x2',
    'nonce': '0x7',
    'input': '0x',
    'r': '0xd0d81142192f5f09c930baa437f31c95786b45bfd54796631435b7998cd937ea',
    's': '0x79133f8bdc1cb5aa9d568cbaa24bf491c8368b25311db0475f241f41261e4670',
    'chainId': '0x1',
    'v': '0x1',
    'gas': '0x5208',
    'maxPriorityFeePerGas': '0xb165100c3',
    'from': '0xa541a021235c6ef90ec4ad4eac2e1bbbb39bacd7',
    'to': '0x0b21c2dc3866aa7f5624578ae612758611d27401',
    'maxFeePerGas': '0xb165100c3',
    'value': '0x16ecdf982bbdde8',
    'gasPrice': '0xb165100c3'},
   {'blockHash': '0xf26f3d8f69a

In [1]:
from solana.keypair import Keypair

# Generate a new keypair
keypair = Keypair.generate()

# Public and private keys
public_key = keypair.public_key
private_key = keypair.secret_key

print(f"Public Key: {public_key}")


ModuleNotFoundError: No module named 'solana.keypair'

# Start Tracking

In [ ]:

currency = arr[r]


{'id': 'sportsology-game', 'symbol': 'game', 'name': '$GAME Token'}

# Apply (ddy - dy)